In [ ]:
!pip install syft==0.2.9 >/dev/null

ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: google-colab 1.0.0 has requirement notebook~=5.3.0; python_version >= "3.0", but you'll have notebook 5.7.8 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 4.5.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: bokeh 2.1.1 has requirement tornado>=5.1, but you'll have tornado 4.5.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


Both the data and model should only reside on bob's machine

In [ ]:
import torch
import syft as sy

hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id='bob')

# Creating dummy data and placing it at bob
input = torch.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = torch.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

weights = torch.tensor([[0.], [0.]], requires_grad=True).send(bob)
pred = input.mm(weights)
loss = ((pred - target)**2).sum()

loss.backward()

weights.data.sub_(weights.grad * 0.1) # lr = 0.1
weights.grad *= 0 # Clear grad

print(loss.get().data)

tensor(2.)


In [ ]:
# One more iteration
pred = input.mm(weights)
loss = ((pred - target)**2).sum()
loss.backward()
weights.data.sub_(weights.grad * 0.1)
weights.grad *= 0

print(loss.get().data)

tensor(0.5600)


The loss went down!

## Garbage Collection and Common Errors

In [ ]:
bob = sy.VirtualWorker(hook, id='bob')

In [ ]:
bob._objects

{12863331608: tensor([[0.2400],
         [0.6000]], requires_grad=True), 20753985763: tensor([[-0.2000],
         [-0.4000]]), 36677278360: tensor([[1.],
         [1.],
         [0.],
         [0.]], requires_grad=True), 68931993837: tensor([[1., 1.],
         [0., 1.],
         [1., 0.],
         [0., 0.]], requires_grad=True), 80548053162: tensor([[0.6000],
         [0.4000],
         [0.2000],
         [0.0000]], grad_fn=<MmBackward>), 83672260927: tensor([[-0.2000],
         [-0.4000]])}

In [ ]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [ ]:
bob._objects

{}

In [ ]:
x = torch.tensor([1,2,3,4,5]).send(bob)
bob._objects

{74729034632: tensor([1, 2, 3, 4, 5])}

In [ ]:
del x

In [ ]:
bob._objects

{}

Here, if you locally delete the pointer to an object in a remote worker, then it assumes that the object is no longer needed and the object in the remote virtual worker is also deleted.

How to prevent that? make garbage collection for that object False. It is set as true by default.

In [ ]:
x = torch.tensor([1,2,3,4,5]).send(bob)
bob._objects

{31665656076: tensor([1, 2, 3, 4, 5])}

In [ ]:
x.child.garbage_collect_data

True

You can see that it is set as True by default.

In [ ]:
# If you reinitialize x as something else
x = 'gasfgvuystvydg'
bob._objects

{}

The object in the virtual worker got deleted as well since the pointer x no longer points to it.

In [ ]:
## Clearing again
bob = bob.clear_objects()
bob._objects

{}

In [ ]:
x = torch.tensor([1,2,3]).send(bob)
x = torch.tensor([1,2,3]).send(bob)

bob._objects

{212542605: tensor([1, 2, 3])}

There is still one object even after multiple assignment to x. That's because it gets reinitialized.

### Cannot interoperate between a remote tensor and a local tensor

In [ ]:
bob = bob.clear_objects() # Dont forget to call the method as () or it will get initialized to a bound method and will lose the reference to bob
# For example
x = torch.tensor([1,2,4]).send(bob)
y = torch.tensor([1,1,1])

z = x + y

TensorsNotCollocatedException: ignored

### Cannot interoperate between tensors located on different machines

In [ ]:
alice = sy.VirtualWorker(hook, id='alice')

x = torch.tensor([1,2,3]).send(bob)
y = torch.tensor([1,1,1]).send(alice)

z = x + y

TensorsNotCollocatedException: ignored